<a href="https://colab.research.google.com/github/ja8an/big-data/blob/main/SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 199 kB 48.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=fbeef11e5f04288e1ec94a7ecb24b8af77f2d1c9aa50a16242a54966f25238b8
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [4]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [5]:
!wget -P -Q https://media.githubusercontent.com/media/ja8an/big-data/main/in/forestfireinput.csv -O forestfireinput.csv

--2022-11-09 22:07:14--  https://media.githubusercontent.com/media/ja8an/big-data/main/in/forestfireinput.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25425 (25K) [text/plain]
Saving to: ‘forestfireinput.csv’

estfireinput.csv    100%[===================>]  24.83K  --.-KB/s    in 0.002s  

2022-11-09 22:07:14 (13.1 MB/s) - ‘forestfireinput.csv’ saved [25425/25425]



In [6]:
schema = 'x_axis long, y_axis long, month string, day string, FFMC double, DMC double, DC double, temp double, RH double, wind double, rain double, area double'
forestfireinput = spark.read.csv('/content/forestfireinput.csv', schema=schema)

In [7]:
forestfireinput.select(forestfireinput.temp).limit(4).toPandas()

,temp
0,5.1
1,6.7
2,6.7
3,9.0


In [8]:
from pyspark.sql.functions import upper
forestfireinput.withColumn('upper_month', upper(forestfireinput.month)).limit(4).toPandas()

,x_axis,y_axis,month,day,FFMC,DMC,DC,temp,RH,wind,rain,area,upper_month
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51.0,6.7,0.0,MAR
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33.0,0.9,0.0,OCT
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33.0,1.3,0.0,OCT
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97.0,4.0,0.2,MAR


In [9]:
forestfireinput.select("temp").limit(4).toPandas()

,temp
0,5.1
1,6.7
2,6.7
3,9.0


In [10]:
from pyspark.sql.functions import when 
forestfireinput.select("temp", when(forestfireinput.area > 0.0, "Burned").otherwise("Did not burn").alias("Did it burn?")).limit(10).toPandas()

,temp,Did it burn?
0,5.1,Did not burn
1,6.7,Did not burn
2,6.7,Did not burn
3,9.0,Burned
4,9.6,Did not burn
5,14.7,Did not burn
6,8.5,Did not burn
7,10.7,Did not burn
8,7.0,Did not burn
9,7.1,Did not burn


In [11]:
forestfireinput[forestfireinput.month.isin("jan", "feb", "mar")].toPandas()

,x_axis,y_axis,month,day,FFMC,DMC,DC,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51.0,6.7,0.0
1,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97.0,4.0,0.2
2,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99.0,1.8,0.0
3,5,5,mar,sat,91.7,35.8,80.8,7.8,15.1,27.0,5.4,0.0
4,6,4,mar,wed,89.2,27.9,70.8,6.3,15.9,35.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
71,6,4,feb,tue,75.1,4.4,16.2,1.9,5.1,77.0,5.4,0.0
72,2,2,feb,sat,79.5,3.6,15.3,1.8,4.6,59.0,0.9,0.0
73,6,5,mar,mon,87.2,15.1,36.9,7.1,10.2,45.0,5.8,0.0
74,3,4,mar,wed,90.2,18.5,41.1,7.3,11.2,41.0,5.4,0.0


In [12]:
forestfireinput[forestfireinput.month.like("%r%")].toPandas()

,x_axis,y_axis,month,day,FFMC,DMC,DC,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51.0,6.7,0.0
1,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97.0,4.0,0.2
2,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99.0,1.8,0.0
3,5,5,mar,sat,91.7,35.8,80.8,7.8,15.1,27.0,5.4,0.0
4,6,4,mar,wed,89.2,27.9,70.8,6.3,15.9,35.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
58,6,5,mar,mon,87.2,15.1,36.9,7.1,10.2,45.0,5.8,0.0
59,3,4,mar,wed,90.2,18.5,41.1,7.3,11.2,41.0,5.4,0.0
60,6,5,mar,thu,91.3,20.6,43.5,8.5,13.3,27.0,3.6,0.0
61,6,3,apr,sun,91.0,14.6,25.6,12.3,13.7,33.0,9.4,0.0


In [13]:
forestfireinput[forestfireinput.month.like("o%")][forestfireinput.month.like('%t')].toPandas()

,x_axis,y_axis,month,day,FFMC,DMC,DC,temp,RH,wind,rain,area
0,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33.0,0.9,0.0
1,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33.0,1.3,0.0
2,8,5,oct,mon,84.9,32.8,664.2,3.0,16.7,47.0,4.9,0.0
3,6,3,oct,tue,90.6,35.4,669.1,6.7,21.7,24.0,4.5,0.0
4,7,4,oct,fri,90.0,41.5,682.6,8.7,11.3,60.0,5.4,0.0
5,7,3,oct,sat,90.6,43.7,686.9,6.7,17.8,27.0,4.0,0.0
6,4,3,oct,sun,92.6,46.5,691.8,8.8,13.8,50.0,2.7,0.0
7,3,4,oct,sun,92.6,46.5,691.8,8.8,20.6,24.0,5.4,0.0
8,3,5,oct,wed,91.4,37.9,673.8,5.2,15.9,46.0,3.6,0.0
9,2,5,oct,sun,92.6,46.5,691.8,8.8,15.4,35.0,0.9,0.0


In [14]:
forestfireinput.filter(forestfireinput.month == 'oct').limit(1).toPandas()

,x_axis,y_axis,month,day,FFMC,DMC,DC,temp,RH,wind,rain,area
0,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33.0,0.9,0.0


In [15]:
forestfireinput.groupBy("month", "day").avg("temp").limit(10).toPandas()

,month,day,avg(temp)
0,jun,mon,6.566667
1,oct,sun,8.800000
2,sep,fri,10.813158
3,jan,sun,0.000000
4,oct,wed,5.200000
5,apr,sun,9.366667
6,feb,wed,3.200000
7,mar,fri,8.363636
8,sep,thu,9.528571
9,jul,thu,9.633333


In [16]:
from pyspark.sql.functions import avg
forestfireinput.groupBy("month", "day").agg(avg("temp").alias("temperature")).orderBy("temperature", asc=False).toPandas()

,month,day,temperature
0,jan,sun,0.000000
1,nov,tue,1.100000
2,feb,tue,1.900000
3,dec,thu,2.000000
4,feb,sat,2.025000
...,...,...,...
59,aug,wed,13.168000
60,jun,wed,13.566667
61,jun,thu,14.400000
62,aug,tue,14.553571


In [17]:
forestfireinput.groupBy("month").count().orderBy("count", asc=True).toPandas()

,month,count
0,nov,1
1,may,2
2,jan,2
3,apr,9
4,dec,9
5,oct,15
6,jun,17
7,feb,20
8,jul,32
9,mar,54


In [18]:
forestfireinput.filter((forestfireinput.temp > 0.0) | (forestfireinput.DMC > 0)).toPandas()

,x_axis,y_axis,month,day,FFMC,DMC,DC,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51.0,6.7,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33.0,0.9,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33.0,1.3,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97.0,4.0,0.2
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99.0,1.8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32.0,2.7,0.0
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71.0,5.8,0.0
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70.0,6.7,0.0
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42.0,4.0,0.0


In [33]:
!wget -P -Q https://media.githubusercontent.com/media/ja8an/big-data/main/in/transactions_amostra.csv -O transactions_amostra.csv

--2022-11-09 22:44:03--  https://media.githubusercontent.com/media/ja8an/big-data/main/in/transactions_amostra.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353513355 (337M) [text/plain]
Saving to: ‘transactions_amostra.csv’

nsactions_amostra.c 100%[===================>] 337.14M   206MB/s    in 1.6s    

2022-11-09 22:44:11 (206 MB/s) - ‘transactions_amostra.csv’ saved [353513355/353513355]



In [32]:
from pyspark.sql import Window
from pyspark.sql.functions import sum, col, max
schema = 'country string, year string, comm_code long,  commodity string, flow string, price double, weight_kg double, quantity_name string, quantity double, category string'
transactions = spark.read.option("Header", True).csv("/content/transactions_amostra.csv", sep=";", schema=schema)
filter = transactions.filter(col("Year") == 2016)
group = filter.groupBy("Flow", "Commodity").agg(sum("quantity").alias("sum_quantity"))
w = Window.partitionBy("Flow")
partition = group.withColumn("max_quantity", max("sum_quantity").over(w))
result = partition.where(col("max_quantity") == col("sum_quantity"))
result.toPandas()

+------+--------------------+-------------+----------------+
|  Flow|           Commodity| sum_quantity|    max_quantity|
+------+--------------------+-------------+----------------+
|Export|     Band saw blades|     327111.0|6.99847369665E11|
|Export|Degras, residues ...| 1.28789327E8|6.99847369665E11|
|Export|Carrots and turni...| 2.38374431E8|6.99847369665E11|
|Export|Parts & accessori...|      81407.0|6.99847369665E11|
|Export|Natural cork arti...|    3307788.0|6.99847369665E11|
|Export|Acrylic or vinyl ...|  5.6025744E7|6.99847369665E11|
|Export|  Lead unwrought nes|  4.5266553E7|6.99847369665E11|
|Export|Lamb carcasses an...|    1006717.0|6.99847369665E11|
|Export|Bituminous or oil...|  1.3490006E7|6.99847369665E11|
|Export|Ultramarine and p...|    2804723.0|6.99847369665E11|
|Export|Womens, girls ski...|    1292954.0|6.99847369665E11|
|Export|   Petroleum bitumen| 4.91377529E8|6.99847369665E11|
|Export|Dog or cat food (...|2.132778757E9|6.99847369665E11|
|Export|Electric table, 

In [ ]:
schema = 'MS Long, Location string, Price double, Bedrooms long, Bathroom long, Size double, Price_SQ_FT double, Status string'
spark.read.option("Header", True).csv("/content/RealState")